# Gen AI Demo

__Import OpenAI key__

In [ ]:
from secret_key import openai_api_key, google_api_key

__Import OpenAI, ChatOpenAI, PromptTemplate, ChatPromptTemplate__

In [ ]:

%pip install -qU langchain-core
%pip install -qU langchain-community
%pip install -qU langchain-openai
%pip install -qU langchain-google-genai
%pip install -qU langchain

%pip install -qU hdbcli
%pip install -qU google-search-results
%pip install -qU numexpr
%pip install -qU pypdf

In [ ]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

from langchain_google_genai import ChatGoogleGenerativeAI

__Use LangChain OpenAI, LangChain ChatOpenAI__

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", openai_api_key=openai_api_key, temperature=0.7)

llm.invoke("How far is the moon away from earth in miles ?")

In [ ]:
chat_llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=openai_api_key, temperature=0.7)
messages = [
        SystemMessage(content="You are an expert in Python programming language"),
        HumanMessage(content="Tell me about scikit learn in 2 sentences")
]
                                     
chat_llm.invoke(messages)

In [ ]:
messages = [
        SystemMessage(content="You are an expert in Python programming language"),
        HumanMessage(content="Tell me about scikit learn in 2 sentences"),
        AIMessage(content="Scikit-learn is a popular machine learning library in Python that provides a wide range of tools for building and deploying machine learning models. It includes various algorithms for classification, regression, clustering, dimensionality reduction, and more, making it a versatile and powerful tool for data scientists and machine learning practitioners."),
        HumanMessage(content="Is there a Scikit-learn library for JavaScript ?")
]
                                     
chat_llm.invoke(messages)

In [ ]:
google_chat_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=google_api_key, temperature=0.7)
messages = [
        SystemMessage(content="You are an expert in Python programming language"),
        HumanMessage(content="Tell me about scikit learn in 2 sentences")
]

google_chat_llm.invoke(messages)

__ChatPromptTemplate with dynamic values__

In [ ]:
chat_prompt_template = ChatPromptTemplate(
    messages = [
        SystemMessagePromptTemplate.from_template("You are an expert in {main_topic}"),
        HumanMessagePromptTemplate.from_template("Tell me about {sub_topic} in 2 sentences")
])

# chat_prompt_template.invoke({"main_topic": "Node.js", "sub_topic": "closures"})

# prompt = chat_prompt_template.invoke({"main_topic": "Node.js", "sub_topic": "closures"})
# chat_llm.invoke(prompt)
                                     
chain = chat_prompt_template | chat_llm
chain.invoke({"main_topic": "Node.js", "sub_topic": "closures"})

In [ ]:
chain.invoke({"main_topic": "SAP Integration Suite", "sub_topic": "OData Adapter"})

__Output Parsers - Format the output for APIs__

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = chat_prompt_template | chat_llm | output_parser
chain.invoke({"main_topic": "United States", "sub_topic": "Atlanta"})

# result = chain.invoke({"main_topic": "Node.js", "sub_topic": "closures"})
# print(type(result))

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

chat_prompt_template = ChatPromptTemplate(
    messages = [
        SystemMessagePromptTemplate.from_template("You have extensive knowledge about all the countries in the world"),
        HumanMessagePromptTemplate.from_template("Name 5 big cities in the following country: {country}")
])
                                     

chain = chat_prompt_template | chat_llm | output_parser
# chain.invoke({"country": "Sweden"})

result = chain.invoke({"country": "Sweden"})
print(type(result))

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class Employee(BaseModel):
    name: str = Field(description="Name of person")
    company: str = Field(description="Name of company")
    experience: int = Field(description="Number of years with company")

output_parser = JsonOutputParser(pydantic_object=Employee)

chat_prompt_template = ChatPromptTemplate(
    messages = [
        SystemMessagePromptTemplate.from_template("Create a fictitious employee with the following name.  \nFormatting Instructions: {format_instructions}"),
        HumanMessagePromptTemplate.from_template("Employee name: {name}")
])
                                     

chain = chat_prompt_template | chat_llm | output_parser
chain.invoke({
    "name": "Milton",
    "format_instructions": output_parser.get_format_instructions()
    })

# result = chain.invoke({
#     "name": "Milton",
#     "format_instructions": output_parser.get_format_instructions()
#     })
# print(type(result))

__Summarization Techniques: Stuff Chain, Map Reduce, Refine__

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import create_retrieval_chain

from langchain.document_loaders import TextLoader
my_character = TextLoader('my_character.txt').load()

In [ ]:
summarize_chain = load_summarize_chain(llm=chat_llm, chain_type="stuff")
summary = summarize_chain.invoke(my_character)

print(summary["output_text"])

In [ ]:
map_template_string = """
Write a concise summary of the following text delimited by 3 backticks.

text: ```{text}```
       
SUMMARY:
"""

map_prompt_template = PromptTemplate(template=map_template_string, input_variables=["text"])


combine_template_string = """
Provide a final concise summary of the following text delimited by 3 backticks.
       
text: ```{text}```
"""
combine_prompt_template = PromptTemplate(template=combine_template_string, input_variables=['text'])

summarize_chain = load_summarize_chain(llm=chat_llm, chain_type="map_reduce", map_prompt=map_prompt_template, combine_prompt=combine_prompt_template)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

documents = text_splitter.split_documents(my_character)

# print(type(documents[0]))

summary = summarize_chain.invoke(documents)
print(summary["output_text"])


In [ ]:
# refine summarization - using the default prompt

summarize_chain = load_summarize_chain(llm=chat_llm, chain_type="refine")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

documents = text_splitter.split_documents(my_character)

# print(type(documents[0]))

summary = summarize_chain.invoke(documents)
print(summary["output_text"])


__Retrieval Augmented Generation (RAG)__

In [ ]:
import os
import json

from hdbcli import dbapi
from langchain_community.vectorstores.hanavector import HanaDB
from langchain.chains.combine_documents import create_stuff_documents_chain

with open("hana_keys.json", "r") as key_file:
    svcKey = json.load(key_file)
    
    
env_vars = {
    "HANA_USER": svcKey["hana_user"],
    "HANA_PASSWORD": svcKey["hana_password"],
    "HANA_HOST": svcKey["hana_host"]
}

os.environ.update(env_vars)

HANA_USER = os.getenv('HANA_USER')
HANA_PASSWORD = os.getenv('HANA_PASSWORD')
HANA_HOST = os.getenv('HANA_HOST')
connection = dbapi.connect(
    address=HANA_HOST,
    port=443,
    user=HANA_USER,
    password=HANA_PASSWORD,
    encrypt='true',
    autocommit=True
)

In [ ]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=openai_api_key)
db = HanaDB(embedding=embedding_model, connection=connection, table_name="GENAI_RAG_DEMO")


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
split_texts = text_splitter.split_text(my_character[0].page_content)

metadata = [
        {
            "source": "My Character",
            "chunk_index": i
        }
        for i in range(len(split_texts))
    ]

db.delete(filter={"source": "My Character"})
db.add_texts(split_texts, metadatas=metadata)

In [ ]:
question_prompt_template = """
Use only the provided context below delimited by 3 backticks to answer the question at the end delimited by 3 backticks.
If you don't know the answer, please reply that the question is not relevant to the character

context: ```{context}```

question: ```{input}``` 
"""

question_prompt = ChatPromptTemplate.from_template(template=question_prompt_template)
question_answer_chain = create_stuff_documents_chain(chat_llm, question_prompt)

retriever = db.as_retriever(filter={"source": "My Character"})
retrieval_chain = create_retrieval_chain(retriever, question_answer_chain)

result = retrieval_chain.invoke({"input": "What was her job ?"})
print(result["answer"])


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("my_man_jeeves.pdf")
documents = loader.load()

# len(documents)
# documents[123]

In [ ]:
metadata = [
        {
            "source": "My Man Jeeves",
            "chunk_index": i
        }
        for i in range(len(documents))
    ]


db.delete(filter={"source": "my_man_jeeves.pdf"})
db.add_documents(documents)

In [ ]:
prompt = "What is Corky's profession ?"
docs = db.similarity_search(prompt, filter={"source": "my_man_jeeves.pdf"}, k=5)

for doc in docs:
    print("-" * 80)
    print(doc.page_content)

In [ ]:
question_prompt_template = """
Use only the provided context below delimited by 3 backticks to answer the question at the end delimited by 3 backticks.
If you don't know the answer, please reply that the question is not relevant to the character

context: ```{context}```

question: ```{input}``` 
"""

question_prompt = ChatPromptTemplate.from_template(template=question_prompt_template)
question_answer_chain = create_stuff_documents_chain(chat_llm, question_prompt)

retriever = db.as_retriever(filter={"source": "my_man_jeeves.pdf"})
retrieval_chain = create_retrieval_chain(retriever, question_answer_chain)

result = retrieval_chain.invoke({"input": "What is Corky's profession ?"})
print(result["answer"])


__Agents__

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

from secret_key import serpapi_api_key


In [ ]:
toolkit = load_tools(["serpapi", "llm-math"], llm=chat_llm, serpapi_api_key=serpapi_api_key)

In [ ]:
agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)
response = agent({"input":"Who is the current President of Japan and what is his age ?"})


In [ ]:
response = agent({"input":"What is the square of the age difference between current President of Japan and Shahrukh Khan ?"})